在window上折腾了一晚上，被小伙伴嘲笑，说谁还在window下搞spark分布式啊。于是又花了差不多一上午的时间在ubuntu下搭了个spark环境，并将其和ipython上交互。以后还是在ubuntu下学习吧，那些命令不用就特别容易忘记。
配置spark开发环境参考的两个主要博客如下，照着这两个博客下去，没有什么坑，亲测有效.命令行中直接输入pyspark进入jupyter notebook环境

[ubuntu spark环境搭建](http://blog.csdn.net/u010171031/article/details/51849562)

[ubuntu anaconda ipython 配置](http://blog.csdn.net/duxu24/article/details/53587451)

[降低pyspark输出日志级别](http://blog.csdn.net/huanbia/article/details/51315889)

我的spark　scala jdk 安装目录在/usr/lib/spark　　/scala  /jdk 下，记一下，免得以后想用忘记在哪了

期间碰到的一些的忘记的linux命令
+ 将一个文件由一个文件夹移入另一个文件夹：在待移文件夹路径下　sudo　mv file 直接写待移到的路径下
+ 给某个文件赋予权限　sudo chmod 777 file
+ 解压zip文件　sudo unzip file

In [11]:
# 开发环境测试
lines = sc.textFile('/home/lee/test.txt')#代表文件中各行文本的RDD 
bLines = lines.filter(lambda line:'b' in line)
print 'this is a transform',bLines
print lines.count()
print bLines.first()
print lines.first()
sc

this is a transform PythonRDD[30] at RDD at PythonRDD.scala:43
5
bbb
aaa


## spark　核心概念简介
＋　**driver program** 驱动器程序 用来发起集群上的各种并行操作。包含应用的main函数，并且定义了集群上的分布式数据集，还对这些分布式数据集应用了相关操作

+ **SparkContext** 驱动器程序通过SparkContext对象来访问Spark.这个对象代表对计算集群的一个连接。shell启动时已经自动创建了一个SparkContext对象，即sc

+ **RDD** 在SparkContext基础上创建起来的。

+ **Spark编程的核心概念**　通过一个驱动程序创建一个SparkContext和一系列RDD,然后进行并行操作　

## 初始化SparkContext
+ Step1: 先创建一个SparkConf对象配置应用
+ Step2:　基于Step1的SparkConf创建一个SparkContext对象


In [8]:
# 交互环境下无需这样　不然会报错
from pyspark import SparkConf,SparkContext
conf = SparkConf().setMaster('local').setAppName('My_App')
sc = SparkContext(conf = conf)

## 构建独立应用
这个只有scala java版，不知道为什么没有python版

## RDD 弹性分布式数据集
RDD可以看成一个简单的“数组”，与一般数组的区别是：RDD是分布式存储，可以更好的利用现有的云数据平台，并在内存中运行。简单的说RDD就是分布式的元素集合（不可变的分布式对象集合）。在Spark中，对数据的所有操作不外乎创建RDD　转化已有RDD以及调用RDD操作进行求值。Spark会自动将RDD中的数据分发到集群上，并将操作并行化执行
### 1. 创建RDD
+ 读取一个外部数据集
+ 在驱动器程序里分发驱动器程序的对象集合（list set）进行并行化，这是RDD创建最简单的一个方式，即将一个已有的集合传给SparkContext的parallelize()方法。这个一般用在开发原型和测试时，因为这种方式需要把整个数据集先放在一台机器的内存中
### 2. RDD两种类型的操作
+ transformation 转化操作　由一个RDD生成一个新的RDD,转化出来的RDD是惰性求值的，只有在行动操作中用到这些RDD时才会被计算。如map() filter(),返回值类型是RDD
+ action 行为操作 会对RDD计算出一个结果，必然会用到RDD转化操作。并将结果返回到驱动器程序中，或把结果存储到外部存储系统中（HDFS ）,会触发实际计算，如count() first(),返回的是其他数据类型

**注意**：虽然可以在任何时候定义新的RDD,但spark只会惰性计算这些RDD.默认情况下，Spark的RDD会在每次对他们进行行动操作时重新计算。若想在多个行动中重用同一个RDD,可以使用RDD.persist()让spark把这个RDD缓存下来（放在内存中）。

### 3. 惰性求值
意味着在被调用行动操作之前Spark不会开始计算。Spark会在内部记录下所要求执行的操作相关信息，从这个角度看，我们可以把RDD看作我们通过转化操作构建出来的　记录如何计算数据的指令列表。把数据读取到RDD的操作同样也是惰性的。

**优势**　可以把一些操作合并到一起来减少计算数据的步骤。

### 4. 向Spark传递函数
Spark的大部分转化操作和一部分行动操作，都需要依赖用户传递函数来计算。python中的三种函数传递方式
+ lambda表达式　传递比较短的函数
+ 传递顶层函数
+ 定义局部函数

### 5. 常见的转化操作和行动操作
#### 基本RDD (支持任意RDD数据类型)
+ 针对各个元素的转化操作
  + map()
  + flatMap() 将函数应用于RDD 中每个元素，将返回的迭代器所有内容展平，构成新的RDD,通常用来切分单词
+ 伪集合操作　支持合并于相交，但要求操作的RDD是相同的数据类型

  + distinct() rdd1.distinct() 生成一个只包含不同元素的新RDD 但操作开销很大，因为会将所有数据通过的网络进行混洗
  + union() rdd1.union(rdd2) 并集操作,与数学中不同的是，即使是相同的元素也会重复包含在并集中
  + intersection()　rdd1.intersection()　交集操作　去除重复元素，效率很差，也要经过混洗
  + substract()　rdd1.subtract(rdd2) 差集操作　也需要数据混洗
  + cartesian() rdd1.cartesian(rdd2) 笛卡尔积　返回所有可能的（a,b）对，在考虑所有可能组合的相似度时比较有用，但在大规模RDD的笛卡尔积开销巨大
+ 行动操作
  + reduce() 并行整合RDD中所有数据　接受一个函数作为参数，这个函数要操作两个RDD的元素类型的数据并返回一个同样类型的新元素。常用来对RDD中的元素进行累加
  + fold() 和reduce一样，但需要提供初始值　接受一个与reduce()接收的函数签名相同的函数，再加上一个’初始值‘来作为每个分区第一次调用时的结果。所提供的初始值应当是提供操作的单位元素，即使用你的函数对这个初始值进行多次计算不会改变结果
  + aggregate() 和reduce类似，但是通常返回不同类型的函数。需要提供我们期待返回的类型的初始值，然后通过一个函数吧RDD中的元素合并起来放入累加器。考虑到每个节点是在本地进行累加，最终，还需要把第二个函数将累加器两两合并
  + countByValue() 各元素在RDD中出现的次数，以字典的形式返回
  + top(n) 测试表明返回排序后的top(n)
  + takeOrdered(num)　从rDD中按照提供的顺序返回最前面的num个元素
  + foreach() 对RDD中所有元素应用一个行动操作，但是不把任何结果返回到驱动器程序中
+ 不同RDD 类型间的转换
mean() variance()只能用在数值RDD上，join()只能用在键值对RDD上

### 持久化
在python中，始终会序列化要持久化存储的数据，所以持久化级别默认值就是以序列化后的对象存储在JVM堆空间中。当我们把数据写到磁盘或者堆外存储上时，也总是使用序列化后的数据

In [10]:
# parallelize()方法
lines = sc.parallelize(['pandas','i love pandas'])
lines.first()

'pandas'

In [18]:
# lambda 函数
inputRDD = sc.textFile('/home/lee/test.txt')
errorRDD = inputRDD.filter(lambda x: 'error' in x)
#print errorRDD.count()
warningRDD = inputRDD.filter(lambda x: 'warning' in x)
#warningRDD.count()
badLinesRDD = errorRDD.union(warningRDD)
badLinesRDD.count()

# 使用take获取RDD中少量元素
for line in badLinesRDD.take(5):
    print line
    
# collect()用来获取整个RDD中数据　不能用在大规模数据集上，只有当数据集能在单台机器的内存中放得下的时候
# 大多数情况下，RDD不能通过collect()收集到驱动程序中，此时，我们通常把数据写到诸如HDFS 这样的分布式存储系统中
# 可以使用saveAsTextFile() saveAsSequenceFile(),或者任意其他行动操作把RDD数据内容以各自自带的格式保存起来


# 自定义函数
def containsError(s):
    return 'error' in s
word = rdd.filter(containsError)

# 局部函数　
# 传递函数时需要小心一点，python会在不经意间把函数所在的对象也序列化传出去。当你传递的对象是某个对象的
# 成员时，或者包含老对某个对象中一个字段的引用时，Spark就会把整个对象发到工作节点上，这可能比你想
# 传递的东西大得多

# 传递一个带字段引用的函数的正确打开方式
class SearchFunctions(objects):
    def __init__(self,query):
        self.query = query
    def isMatch(self,s):
        return self.query in s
    def getMatchesNoReference(self,rdd):
        query = self.query # 非常重要，一定要把所需字段从对象中拿出来放到一个局部变量中，然后传递这个局部变量
        return rdd.filter(lambda x : self.query in x)


aaa,error
cccerror
error,error
bad error
error,warning


In [46]:
# 5中转化操作和行动操作的example
## 5.1 map
nums = sc.parallelize([1,2,3,4])
squared = nums.map(lambda x: x*x).collect()
for num in squared:
    print '%i'%num
    
## 5.1flatMap() 对每个输入元素生成多个输出元素　flatMap函数被分别应用到输入RDD的每个元素上
## 返回一个返回值序列的迭代器，即将各个迭代器均展平，得到一个包含各个迭代器可访问的所有元素的RDD 
## 常用来将输入的字符串切分为单词
lines = sc.parallelize(['hello spark','hi' ,'Hi,spark'])
words = lines.flatMap(lambda line:line.split(' ')).collect()
for word in words:
    print word
#words.first()

# rdd = sc.parallelize([1,2,3,3])
# # ss = rdd.flatMap(lambda x:x.to(3)).collect()#报错了。。有经验再回来研究
# # for x in ss:
# #     print x

# ss = rdd.sample(False,0.5).collect()
# for i in ss:
#     print i #没有输出。。

rdd = sc.parallelize([19,1,7,4,5])
sum1 = rdd.reduce(lambda x,y:x+y)
sum2 = rdd.fold((2),lambda x,y:x+y)# 都不知道如何查文档。。为什么会这样？
sum2
#rdd.top(3)
rdd.takeOrdered(2) # 什么鬼　我要查文档。。

1
4
9
16
hello
spark
hi
Hi,spark


[1, 4]

True

## 键值对操作
键值对RDD是spark中许多操作所需要的常见数据类型，通常用来做聚合计算。一般通过一些初始的ETL操作将数据转化为键值对形式。spark为包含键值对类型的RDD提供了一些专有的操作，这些RDD被称为pair RDD

### 创建Pair RDD
在Python中，为了让提取键之后的数据能够在函数中使用，需要返回一个由二元组组成的RDD
```python
pairs = lines.map(lambda x: (x.split(' ')[0],x))
```

### Pair RDD 的转化操作
Pair RDD 可以使用所有标准RDD上的可用的转化操作，但由于Pair RDD中包含二元组，所以需要传递的函数应当操作二元组而不是独立的元素

+ reduceByKey(func) 合并具有相同建的值
```python
rdd.reduceByKey(lambda x,y:x+y)
```
+ groupByKey() 对具有相同建的值进行分组
+ combineByKey(createCombiner,mergeValue,mergeCombiners,partitioner)　使用不同的返回类型合并具有相同键的值
在处理数据时是处理每一个元素，即遍历分区中所有元素。若某元素在前面的遍历中未出现过，combineByKey()会使用一个叫做createCombiner()的函数创建那个键对应累加器的初始值。需要注意的是，这一过程会在每个分区中第一次出现各个键时发生，而不是整个RDD中第一次出现一个键时发生如果这是一个在处理当前分区之前遇到的键，他会使用mergeValue()方法将该键的累加器对应的当前值与这个新值进行合并；由于每个分区都是独立处理的，因此对于同一个键可以有多个累加器。如果有两个或者更多的分区都有对应同一个键的累加器，就需要使用用户提供的mergeCombiners()方法将各个分区的结果进行合并。
+ mapValues(func) 对pair RDD中的每个值应用一个函数而不改变键
+ flatMapValues(func) 对pair RDD 中的每个值应用一个返回迭代器的函数，然后对返回的每个元素都生成一个对应原键的键值对记录，通常用于符号化
+ keys()　返回一个仅包含键的RDD
+ values() 返回一个仅包含值的RDD
+ sortByKey() 返回一个根据键排序的RDD

**针对两个pair RDD的操作**
+ substractByKey 删掉RDD中键与other RDD中的建相同的元素
+ 对两个RDD进行内连接
+ rightOuterJoin 对两个RDD进行链接操作，确保第一个RDD的键必须存在（右外连接）
+ leftOuterJoin　对两个RDD进行链接操作，确保第二个RDD的键必须存在（左外连接）
+ cogroup 将两个RDD中拥有相同键的数据分组到一起




In [6]:
# 用python对第二个元素进行筛选
result = pairs.filter(lambda keyValue: len(keyValue[1]<20))

# 利用reduceByKey() 和 mapValue()计算每个键对应的平均值
rdd.mapValue(lambda x:(x,1)).reduceByKey(lambda x,y:(x[0]+y[0]),x[1]+y[1]))

# 单词计算
rdd = sc.textFile('/home/lee/test2.txt')
words = rdd.flatMap(lambda x: x.split(' '))
result = words.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
for i in result.collect():
    print i

result = rdd.flatMap(lambda x: x.split(' ')).countByValue()
result


defaultdict(int,
            {u'': 1,
             u'and': 2,
             u'dad': 1,
             u'dad.': 1,
             u'do': 1,
             u'i': 1,
             u'love': 3,
             u'me,too.': 1,
             u'me?': 1,
             u'mom': 2,
             u'mom,': 1,
             u'my': 1,
             u'you': 1})

In [ ]:
# 并行度调优　在执行聚合或分组操作时，可以要求spark使用给定的分区数
data = [('a',3),('b',4),('a',1)]
sc.parallelize(data).reduceByKey(lambda x,y:x+y,10) #自定义并行度为１０

# python查看分区数
rdd.getNumPartitions

# 在python中以字符串顺序对整数进行自定义排序
rdd.sortByKey(ascending=True,numPartitions=None,keyfunc = lambda x :str(x))

## Pair RDD行动操作
+　countByKey() 对每个键对应的元素分别计数
+ collectAsMap()　将结果以映射表的形式返回，以便查询
+ lookup(key)　返回给定键对应的所有值

## 数据分区（进阶）(未细看，等把基础知识复习一遍再看)
在分布式程序中，通信的代价是很大的，因此控制数据分布以获得最少的网络传输可以极大的提升整体性能。和单节点的程序需要为记录集合选择合适的数据结构一样,Spark 程序可以通过控制RDD 分区方式来减少通信开销。只有当数据集多次在
诸如连接这种基于键的操作中使用时,分区才会有帮助。

Spark 中所有的键值对 RDD 都可以进行分区，Spark 可以确保同一组的键出现在同一个
节点上。比如,你可能使用哈希分区将一个 RDD 分成了 100 个分区,此时键的哈希值对
100 取模的结果相同的记录会被放在一个节点上。你也可以使用范围分区法,将键在同一
个范围区间内的记录都放在同一个节点上。

为生成结果RDD设好分区方式的操作
+ cogroup()
+ groupWith()
+ join()


## 数据读取与保存
### 3种常见的数据源
+ **文件格式与文件系统**
对于存储在本地文件系统或分布式文件系统（比如NFS、HDFS、Amazon S3 等）中的
数据，Spark 可以访问很多种不同的文件格式，包括文本文件、JSON、SequenceFile，
以及protocol buffer

+ Spark SQL中的结构化数据源
Spark SQL 模块，它针对包括JSON 和Apache Hive 在内的结构化数据
源，为我们提供了一套更加简洁高效的API。

+ Spark 自带的库和一些第三方库
它们可以用来连接Cassandra、HBase、
Elasticsearch 以及JDBC 源。

### 文件格式
Spark 对很多种文件格式的读取和保存方式都很简单
Spark支持的一些常见格式

+ 文本文件 否 普通的文本文件，每行一条记录
+ JSON 半结构化 常见的基于文本的格式，半结构化；大多数库都要求每行一条记录
+ CSV 是 非常常见的基于文本的格式，通常在电子表格应用中使用
+ SequenceFiles 是 一种用于键值对数据的常见Hadoop 文件格式
+ Protocol buffers 是 一种快速、节约空间的跨语言格式
+ 对象文件 是 用来将Spark 作业中的数据存储下来以让共享的代码读取。改变类的时候
 它会失效，因为它依赖于Java 序列化
 
#### 文本文件
在Spark 中读写文本文件很容易。当我们将一个文本文件读取为RDD 时，输入的每一行
都会成为RDD 的一个元素。也可以将多个完整的文本文件一次性读取为一个pair RDD，
其中键是文件名，值是文件内容

**Python中读取一个文本文件**
```python
input = sc.textFile("file:///home/holden/repos/spark/README.md")
```
**Python中将数据保存为文本文件**
```python
result.saveAsTextFile(outputFile)
```

#### JSON
JSON 是一种使用较广的半结构化数据格式。读取JSON 数据的最简单的方式是将数据
作为文本文件读取，然后使用JSON 解析器来对RDD 中的值进行映射操作

**Python中读取JSON**
```python
import json
data = input.map(lambda x: json.loads(x))
```
写出JSON 文件比读取它要简单得多，因为不需要考虑格式错误的数据，并且也知道要写
出的数据的类型

**保存json**
```python
# 选出喜爱熊猫的人
(data.filter(lambda x: x["lovesPandas"]).map(lambda x: json.dumps(x))
.saveAsTextFile(outputFile))
```

#### CSV
读取CSV/TSV 数据和读取JSON 数据相似，都需要先把文件当作普通文本文件来读取数
据，再对数据进行处理。由于格式标准的缺失，同一个库的不同版本有时也会用不同的方
式处理输入数据。

+ CSV 的所有数据字段均没有包含换行符，你也可以使用textFile() 读取并
解析数据

```python
import csv
import StringIO

def loadRecord(line):
"""解析一行CSV记录"""
input = StringIO.StringIO(line)
reader = csv.DictReader(input, fieldnames=["name", "favouriteAnimal"])
return reader.next()
input = sc.textFile(inputFile).map(loadRecord)
```

+ 字段中嵌有换行符，就需要完整读入每个文件，然后解析各段

```python
def loadRecords(fileNameContents):
"""读取给定文件中的所有记录"""
input = StringIO.StringIO(fileNameContents[1])
reader = csv.DictReader(input, fieldnames=["name", "favoriteAnimal"])
return reader
fullFileData = sc.wholeTextFiles(inputFile).flatMap(loadRecords)
```

**python中写csv**

```python
def writeRecords(records):
"""写出一些CSV记录"""
output = StringIO.StringIO()
writer = csv.DictWriter(output, fieldnames=["name", "favoriteAnimal"])
for record in records:
writer.writerow(record)
return [output.getvalue()]
pandaLovers.mapPartitions(writeRecords).saveAsTextFile(outputFile)
```



## Spark编程进阶


## 在集群上运行Spark

## Spark调优与调试

## SparkSQL
Spark SQL 三大功能
+ Spark SQL 可以从各种结构化数据源（例如JSON、Hive、Parquet 等）中读取数据。
+ Spark SQL 不仅支持在Spark 程序内使用SQL 语句进行数据查询，也支持从类似商业
智能软件Tableau 这样的外部工具中通过标准数据库连接器（JDBC/ODBC）连接Spark
SQL 进行查询。
+ 当在Spark 程序内使用Spark SQL 时，Spark SQL 支持SQL 与常规的Python/Java/Scala
代码高度整合，包括连接RDD 与SQL 表、公开的自定义SQL 函数接口等。这样一来，
许多工作都更容易实现了

## Spark Streaming

## 基于Mllib的机器学习
